### Vamos começar a analise exploratória de dados carregando o dataset para criar o dataframe pandas e verificar a quantidade de linhas e colunas

In [ ]:
# Importação das bibliotecas utilizadas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy.stats import mannwhitneyu, chi2_contingency

# Carregando o arquivo csv e criando o DataFrame pandas
df = pd.read_csv('_data/dataset_2021-5-26-10-14.csv', sep= '\t', encoding='utf-8')

# Verificando a quantidade de linhas e colunas do DataFrame
print(f"Quantidade de linhas e colunas: {df.shape}\n")

In [ ]:
# Configuração utlizada para exibir todas as colunas ao executar o df.head()
pd.set_option('display.max_columns', None)

# Ajusta a exibição dos valores para duas casas decimais
pd.options.display.float_format = '{:.2f}'.format

#Exibe as 5 primeiras linas do DataFrame
df.head()

### Sabemos que os valores faltantes estão marcados como missing, então vamos substituir tudo que for missing por nan para facilitar o tratamento a partir dos métodos do Pandas e Numpy

In [ ]:
#Substitui os valores missing por nan
df.replace('missing', np.nan, inplace=True)

# Exibe informações gerais do DataFrame
df.info()

### É possível ver que temos valores faltantes em quatro colunas, então vamos verificar a quantidade ausente em cada

In [ ]:
# Soma a quantidade de valores ausentes em cada coluna e coloca em ordem decrescente
valores_ausentes = df.isna().sum().sort_values(ascending=False)

# Exibe apenas as colunas com valores ausentes e suas respectivas quantidades
print(f"{valores_ausentes[valores_ausentes > 0]} \n")

# Proporção de valores ausentes
print((valores_ausentes[valores_ausentes > 0] / df.shape[0]).apply(lambda x: f"{x:.3f}"))

In [ ]:
df_numerico = df.select_dtypes(include=np.number)

# Contando valores menores que zero por coluna
negativos_por_coluna = (df_numerico < 0).sum()

print("Quantidade de valores menores que zero por coluna:")
print(negativos_por_coluna)

### Aqui foram identificados 144 valores negativos em valor_total_pedido, o que não faz sentido, então esses valores deverão ser tratados posteriormente

In [ ]:
# Verificar valores unicos em cada variavel
valores_unicos = []
print("Quantidade de valores únicos em cada coluna:")
for col in df.columns.tolist():
    print(f"{col} : {len(df[col].value_counts())}")    

### A variável "participacao_falencia_valor" só possui um valor para todo o dataset, dessa forma já vamos descartá-la de outras análises, pois tendo apenas um valor ela não ajuda a resolver o problema de classificação

### Verificando a quantidade de valores unicos em cada variável é possivel notar que dentre as categóricas temos duas com grandes quantidades de categoria, o que pode ser problemático, pois elas resultariam num aumento da dimensionalidade e com isso prejudicar a performance do modelo

In [ ]:
# Principais medidas estatísticas das variáveis numéricas
df.describe().T

### Verificando as variáveis continuas vemos que muitas delas possuem grande parte dos valores concentrados em zero, mas com valores máximos bem distantes, ou seja, há indicio da presença de muitos outliers que serão analisados e tratados para reduzir seu impacto no modelo

In [ ]:
# Estatísticas para variáveis categóricas
df.describe(include=["object"]).T.round(2)

### Nesta seção vamos análisar a proporção da variável alvo com relação a quantidade de linhas com valores nulos para saber se a diferença entre as linhas totais e a quantidade ao excluir as linhas com NaNs é estatisticamente significativa

In [ ]:
alvo = "default"

print(f"Distribuição total \n {df[alvo].value_counts(), df[alvo].value_counts(normalize=True)} \n")

# separa grupos
df_nan = df[df.isna().any(axis=1)]      # linhas com pelo menos 1 NaN
df_no_nan = df[~df.isna().any(axis=1)]  # linhas sem nenhum NaN

# contagem absoluta
print(f"Com NaN:\n {df_nan[alvo].value_counts(), df_nan[alvo].value_counts(normalize=True).apply(lambda x: f"{x:.3f}")} \n")

# Proporção
print(f"Sem NaN:\n {df_no_nan[alvo].value_counts(), df_no_nan[alvo].value_counts(normalize=True).apply(lambda x: f"{x:.3f}")} \n")


In [ ]:
# Contar o número total de linhas duplicadas
num_duplicadas = df.duplicated().sum()
print(f"Número de linhas duplicadas: {num_duplicadas}")

In [ ]:
# Contagem total da variavel alvo
total_counts = df[alvo].value_counts()

# com e sem NaN
nan_counts = df[df.isna().any(axis=1)][alvo].value_counts()
no_nan_counts = df[~df.isna().any(axis=1)][alvo].value_counts()

# monta tabela de contingência
matriz_de_contingencia = pd.DataFrame({
    "Com_NaN": nan_counts,
    "Sem_NaN": no_nan_counts,
    "Total": total_counts
}).fillna(0).astype(int)

print("Matriz de contingência:")
print(matriz_de_contingencia, "\n")

# Teste qui-quadrado
chi2, p, dof, expected = chi2_contingency(matriz_de_contingencia[["Com_NaN","Sem_NaN"]])

print(f"Qui-quadrado: {chi2:.4f}")
print(f"p-valor: {p:.4f}")


### Como o p valor ficou abaixo de 0.05 há uma significancia estatistica de associação entre as linhas com valores nulos e a variavel alvo

In [ ]:
# Seleciona as variáveis numéricas e exclui "participacao_falencia_valor"
num_cols = df.select_dtypes(include=[np.number]).columns.drop(alvo).drop("participacao_falencia_valor")

# Correlação entre as variáveis
plt.figure(figsize=(15,10))
sns.heatmap(df[num_cols].corr(), annot= True, fmt='.2f')

### Com auxilio dessa matriz podemos ver que nenhuma variável é reduntante (correlação acima de 0.8), principalmente para modelos baseados em árvores, porém vamos realizar uma análise unidimensional para decidir se vamos manter todas as variáveis ou não, pois ao reduzir a quantidade de variáveis diminuimos o custo computacional.

In [ ]:
plt.rcParams["figure.figsize"] = [20.00, 15.00]
plt.rcParams["figure.autolayout"] = True
f, eixos = plt.subplots(4,4)

linha, coluna = 0, 0

for i in num_cols:
    sns.boxplot(data=df,y=i, ax=eixos[linha][coluna])
    coluna += 1
    if coluna == 4:
        linha += 1
        coluna = 0
        if linha == 4:   
            break
                
plt.show()

### Acima podemos ver que há uma grande quantidade de outliers em diversas variáveis então vamos avaliá-las para entender esses valores e decidir como tratar os mesmos. Também vamos aplicar o teste de Mann-Whitney para tentar descobrir quais variáveis tem mais poder discriminatório entre quem dá default e quem não dá

In [ ]:
# Função que recebe uma coluna do df e retorna informações como limite superior e inferior calculado a partir da distancia interquartil,
# assim como também apresenta a quantidade de valores ausentes, resultado de testes estatísticos e graficos referentes a cada variavel.
def analise_continuas(coluna):
    print (f"Coluna: {coluna}")
    print(f"Quantidade de valores unicos: {len(df[coluna].unique())}")
    df_desc = df[coluna].describe().T
    DQ = df_desc["75%"] -  df_desc["25%"]
    limite_inf = df_desc["25%"] - 1.5*DQ
    limite_sup = df_desc["75%"] + 1.5*DQ
    outliers = df[(df[coluna] > limite_sup) | (df[coluna] < limite_inf)] [[coluna, alvo]]
    num_valores_ausentes = df[coluna].isna().sum()
    
    
    print(f"Limite Inferior: {limite_inf}")
    print(f"Limite Superior: {limite_sup}")
    print(f"Quantidade de Outliers: {outliers.shape[0]} ")
    print(f"Número de valores ausentes na variável {coluna}: {num_valores_ausentes} \n")
    
    # Testes estatísticos
    grupo_1 = df [(df[alvo] == 0) & (~df[coluna].isna())][coluna]
    grupo_2 = df [(df[alvo] == 1) & (~df[coluna].isna())][coluna]

    statistic, p_value = mannwhitneyu(grupo_1, grupo_2)
    print(f"p valor do teste de Mann-Whitney : {p_value:.8f} \n\n")
    
    # Graficos
    
    fig = px.histogram(df, x=coluna, color=alvo, nbins=20, barmode="relative", title=f"Distribuição da variável {coluna} por {alvo}")
    fig.update_layout(width=600, height=500)
    fig.show()
    
    plt.figure(figsize=(6, 4))
    sns.scatterplot(data=df, x=coluna, y=alvo)
    plt.show()

In [ ]:
# Chama a função para todas as colunas numericas
for colunas in num_cols:
    analise_continuas(colunas)

### Antes de decidir o que fazer com outliers é uma boa prática consultar a área de negócios para entender se aqueles valores de fato fazem sentido ou se realmente se tratam de erros, porém para resolver o exercícios vamos utilizar a técnica de substituir os outliers pelo limite superior (quando esse limite for diferente de 0) e para os valores negativos vamos substituir por 0. Outra ação que será tomada é a transformação logaritmica das variáveis com valores máximos muito altos para reduzir o peso deles

In [ ]:
# Função que analisa as variáveis categóricas
def analise_categoricas(coluna):
    print(coluna)
    print(f"{str(df[coluna].isna().sum())} ausentes")
    
    matriz_contingencia = pd.crosstab(df[coluna], df[alvo])
    
    chi2, p, dof, expected = chi2_contingency(matriz_contingencia)
    print(f"Teste Qui-Quadrado {chi2}")
    print(f"p valor: {p} \n")
    

In [ ]:
for coluna in df.select_dtypes(include=object):
    analise_categoricas(coluna)

### Para as variáveis categóricas foi aplicado o Teste Qui-Quadrado a fim de descobrir se a associação entre elas e o alvo são estatisticamente significantes, todas tiveram p valor menores que 0.05, mostrando associação com o target

### Como o Teste Mann-Whitney e o Qui-Quadrado não nos ajudaram a escolher quais as variáveis mais importantes, uma vez que o p valor deu menor que 0.05 para todas as variáveis (ocorrencia comum em datasets grandes), vamos analisar a Área sob a curva ROC para ver quais variáveis performam melhor

In [ ]:
import pandas as pd
from sklearn.metrics import roc_auc_score
from scipy.stats import mannwhitneyu

resultados = []

for col in num_cols:
    try:
        # Calcula a area sob a curva
        auc = roc_auc_score(df[alvo], df[col])
            
        resultados.append((col, auc))
        
    except Exception as e:
        print(f"Erro em {col}: {e}")

# Monta DataFrame de resultados
df_resultados = pd.DataFrame(resultados, columns=["variavel", "auc"]) \
                 .sort_values(by="auc", ascending=False)

print(df_resultados.head(16))

### A análise da área sob a curva ROC nos trouxe algumas variáveis que sozinhas tem um poder discriminatório maior, ainda que não muito grande, mas já podemos identificar algumas que devem ter no modelo final, sendo elas as com auc > 0.55. As variáveis com valor = 0.5 não possuem nenhum poder discriminativo univariado, na pratica seria equivalente ao classificar ao acaso, mas combinado com outras variáveis podem ser uteis para o modelo então não serão totalmente descartadas.

## Resumo das análises 

- Dataset grande com 117273 linhas e 22 colunas ao todo, sem linhas duplicadas, 4 variáveis categóricas e 18 contínuas;
- Grande quantidade de Outliers que, quando aplicavel, serão tratados com imputação;
- Variável "participacao_falencia_valor" descartada por possuir apenas um valor em todo o dataset;
- Variáveis contínuas com grande intervalo de valores, necessário aplicar logaritmo para amenizar o impacto de valores extremos;
- Valores negativos na coluna "valor_total_pedido" serão substituidos por 0;
- Os valores ausentes possuem significancia estatistica de associação com a variável "default" então não serão excluídos. É esperado que o algoritmo aprenda se existir alguma relação entre o valor ausente e a variável desfecho;
- Variáveis com maior poder discriminativo: default_3months, ioi_36months, valor_vencido e valor_total_pedido.
